In [120]:
import requests
import base64
import numpy as np
import xarray as xr
import pandas as pd
import ccd
import datetime

In [137]:
import logging
log = logging.getLogger()
log.setLevel(logging.DEBUG)

In [7]:
specs = requests.get('http://localhost:5678/tile-specs').json()
specs_map = dict([ [spec['ubid'], spec ] for spec in specs])

In [20]:
spectral_map = {
    'blue':[
        'LANDSAT_4/TM/sr_band1',
        'LANDSAT_5/TM/sr_band1',
        'LANDSAT_7/ETM/sr_band1',
        'LANDSAT_8/OLI_TIRS/sr_band2'
    ],
    'green':[
        'LANDSAT_4/TM/sr_band2',
        'LANDSAT_5/TM/sr_band2',
        'LANDSAT_7/ETM/sr_band2',
        'LANDSAT_8/OLI_TIRS/sr_band3'
    ],
    'red':[
        'LANDSAT_4/TM/sr_band3',
        'LANDSAT_5/TM/sr_band3',
        'LANDSAT_7/ETM/sr_band3',
        'LANDSAT_8/OLI_TIRS/sr_band4'
    ],
    'nir':[
        'LANDSAT_4/TM/sr_band4',
        'LANDSAT_5/TM/sr_band4',
        'LANDSAT_7/ETM/sr_band4',
        'LANDSAT_8/OLI_TIRS/sr_band5'
    ],
    'swir1':[
        'LANDSAT_4/TM/sr_band5',
        'LANDSAT_5/TM/sr_band5',
        'LANDSAT_7/ETM/sr_band5',
        'LANDSAT_8/OLI_TIRS/sr_band6'
    ],
    'swir2':[
        'LANDSAT_4/TM/sr_band7',
        'LANDSAT_5/TM/sr_band7',
        'LANDSAT_7/ETM/sr_band7',
        'LANDSAT_8/OLI_TIRS/sr_band7'
    ],
    'thermal':[
        'LANDSAT_4/TM/toa_band6',
        'LANDSAT_5/TM/toa_band6',
        'LANDSAT_7/ETM/toa_band6',
        'LANDSAT_8/OLI_TIRS/toa_band10'
    ],
    'cfmask':[
        'LANDSAT_4/TM/cfmask',
        'LANDSAT_5/TM/cfmask',
        'LANDSAT_7/ETM/cfmask',
        'LANDSAT_8/OLI_TIRS/cfmask'
    ]}

In [19]:
numpy_type_map = {
    'UINT8'   : np.uint8,
    'UINT16'  : np.uint16,
    'INT8'    : np.int8,
    'INT16'   : np.int16
}

In [129]:
def ordinal_date(dtstr):
    _dt = datetime.datetime.strptime(dtstr, '%Y-%m-%dT%H:%M:%SZ')
    return _dt.toordinal()

In [107]:
def as_numpy_array(tile):
    """Oh my goodness."""
    spec       = specs_map[tile['ubid']]
    np_type    = numpy_type_map[spec['data_type']]
    shape      = specs_map[spec['ubid']]['data_shape']
    buffer     = base64.b64decode(tile['data'])
    return np.frombuffer(buffer, np_type).reshape(*shape)

In [131]:
def landsat_dataset(spectrum, x, y, t, ubid):
    """Oh my damn."""
    tile_url = 'http://localhost:5678/tiles'
    query    = {'ubid':ubid, 'x':x, 'y':y, 'acquired':t}
    tiles    = requests.get(tile_url, params = query).json()
    rasters  = xr.DataArray([as_numpy_array(tile) for tile in tiles])

    ds = xr.Dataset()
    ds[spectrum] = (('t','x','y'), rasters)
    ds[spectrum].attrs = {'color':spectrum}
    ds.coords['t'] = (('t'), pd.to_datetime([t['acquired'] for t in tiles]))
    ds.coords['source'] = (('t'), [t['source'] for t in tiles])
    ds.coords['acquired'] = (('t'), [t['acquired'] for t in tiles])
    ds.coords['ordinal'] = (('t'), [ordinal_date(t['acquired']) for t in tiles])
    return ds

In [116]:
def rainbow(x,y,t):
    """I wish you could smell what I'm smelling."""
    ds = xr.Dataset()
    
    for (spectrum, ubids) in spectral_map.items():
        for ubid in ubids:
            band = landsat_dataset(spectrum, x, y, t, ubid)
            if band:
                ds = ds.merge(band)
     
    return ds

In [123]:
def detect(rainbow, x, y):
    """The fry bites back."""
    return ccd.detect(blues    = np.array(rainbow['blue'].values[:,x,y]),
                      greens   = np.array(rainbow['green'].values[:,x,y]),
                      reds     = np.array(rainbow['red'].values[:,x,y]),
                      nirs     = np.array(rainbow['nir'].values[:,x,y]),
                      swir1s   = np.array(rainbow['swir1'].values[:,x,y]),
                      swir2s   = np.array(rainbow['swir2'].values[:,x,y]),
                      thermals = np.array(rainbow['thermal'].values[:,x,y]),
                      quality  = np.array(rainbow['cfmask'].values[:,x,y]),
                      dates    = list(rainbow['ordinal']))

In [122]:
# not yet...
# dataset_2 = landsat_dataset('', 1823415, 2342805, '1980-01-01/2020-01-01')

In [132]:
dataset = rainbow(1931415, 2414805, '1980-01-01/2020-01-01')

In [133]:
dataset

<xarray.Dataset>
Dimensions:   (t: 386, x: 100, y: 100)
Coordinates:
  * t         (t) datetime64[ns] 1982-12-31 1983-02-01 1984-06-19 1984-09-07 ...
    source    (t) object 'LT040120301982365-SC20170107023552' ...
    acquired  (t) object '1982-12-31T00:00:00Z' '1983-02-01T00:00:00Z' ...
    ordinal   (t) float64 7.239e+05 7.239e+05 7.244e+05 7.245e+05 7.246e+05 ...
Dimensions without coordinates: x, y
Data variables:
    blue      (t, x, y) float64 456.0 411.0 342.0 338.0 408.0 442.0 340.0 ...
    swir2     (t, x, y) float64 1.087e+03 549.0 202.0 145.0 519.0 736.0 ...
    nir       (t, x, y) float64 1.619e+03 816.0 467.0 954.0 1.636e+03 ...
    green     (t, x, y) float64 686.0 459.0 301.0 437.0 592.0 625.0 565.0 ...
    cfmask    (t, x, y) object 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
    red       (t, x, y) float64 767.0 572.0 425.0 436.0 544.0 529.0 486.0 ...
    swir1     (t, x, y) float64 1.893e+03 798.0 416.0 785.0 1.314e+03 ...
    thermal   (t, x, y) float64 2.696

In [97]:
dataset['red'].values[0:10,0,0]

array([  767.,  1632.,   566.,   431.,  2828.,   815.,  3099.,  3501.,
         899.,  1691.])

In [100]:
dataset['blue'].values[0:10,0,0]

array([  456.,  1775.,   557.,   286.,  2766.,   472.,  3076.,  2978.,
         572.,  1557.])

In [135]:
dataset['ordinal'].values[0:10]

array([ 723910.,  723942.,  724446.,  724526.,  724574.,  724606.,
        724654.,  724670.,  724702.,  724750.])

In [104]:
dataset['cfmask'].values[0:10,0,0]

array([0, 2, 0, 0, 4, 0, 4, 3, 0, 0], dtype=object)

In [ ]:
some_results = [detect(dataset, x, y) for x in range(0, 10) for y in range(0, 10)]

2017-02-05 23:00:05 procedures::fit_procedure        - [50 ]Procedure selected: standard_procedure
2017-02-05 23:00:05 procedures::standard_procedure   - [202]Build change models - dates: 386, obs: (7, 386), meow_size: 12, peek_size: 6
2017-02-05 23:00:05 procedures::standard_procedure   - [226]Processing mask initial count: 134
2017-02-05 23:00:05 procedures::standard_procedure   - [241]Variogram values: [ 121.  126.  162.  538.  256.  180.  540.]
2017-02-05 23:00:05 procedures::standard_procedure   - [246]Initialize for change model #: 1
2017-02-05 23:00:05 change    ::initialize           - [353]Initial slice(0, 12, None)
2017-02-05 23:00:05 change    ::find_time_index      - [163]Sufficient time from times[0..12] (day #723910.0 to #729958.0)
2017-02-05 23:00:05 change    ::initialize           - [363]Checking window: slice(0, 12, None)
2017-02-05 23:00:05 change    ::initialize           - [373]Number of Tmask outliers found: 1
2017-02-05 23:00:05 change    ::initialize           -

In [139]:
some_results[0:10]

[{'algorithm': 'lcmap-pyccd:1.0.3.b1',
  'change_models': [ChangeModel(start_day=729942.0, end_day=736134.0, break_day=736134.0, observation_count=109, change_probability=0, num_coefficients=8, blue=SpectralModel(rmse=134.10760791960541, coefficients=array([ -2.08520106e-02,   1.31907588e+02,   9.61353682e+01,
         0.00000000e+00,   0.00000000e+00,   1.40007186e+01,
         1.13170497e+00]), intercept=15695.423649102591), green=SpectralModel(rmse=141.05397500137212, coefficients=array([ -1.58993391e-02,   4.95715121e+01,   8.07376328e+01,
         0.00000000e+00,   0.00000000e+00,   1.89575333e+01,
         8.06137388e+00]), intercept=12164.591334226905), red=SpectralModel(rmse=142.60971188158152, coefficients=array([ -1.20130970e-02,   2.15136978e+02,   9.54158584e+01,
         0.00000000e+00,   0.00000000e+00,   2.11034024e+01,
         2.14478095e+01]), intercept=9330.8322194775537), nir=SpectralModel(rmse=529.72847277168512, coefficients=array([ -3.31683414e-02,  -1.07025921e+